- https://keras.io/applications/
- https://github.com/yang-zhang/courses/blob/scratch/deeplearning1/nbs/lesson2.ipynb

In [52]:
import math
import os
import datetime

import numpy as np
import pandas as pd

from keras.preprocessing import image
from keras.layers import Input, Lambda, Dense, Dropout, Flatten
from keras.models import Model, Sequential, load_model

from keras.utils import to_categorical
from keras.optimizers import RMSprop, SGD

from keras.applications import xception

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score

from secrets import KAGGLE_USER, KAGGLE_PW

In [53]:
competition_name = 'dog-breed-identification'
data_dir = '/opt/notebooks/data/' + competition_name + '/preprocessed'

gen = image.ImageDataGenerator()
batch_size = 16
target_size=(299, 299)

def add_preprocess(base_model, preprocess_func, inputs_shape=(299, 299, 3)):
    inputs = Input(shape=inputs_shape)
    x = Lambda(preprocess_func)(inputs)
    outputs = base_model(x)
    model = Model(inputs, outputs)
    return model

### train

#### first fine-tune last layer

In [60]:
base_model = xception.Xception(weights='imagenet', include_top=False, pooling='avg')
inputs = Input(shape=(299, 299, 3))
x = Lambda(xception.preprocess_input)(inputs)
x = base_model(x)
outputs = Dense(120, activation='softmax', name='predictions')(x)
model_ft = Model(inputs, outputs)

In [61]:
for layer in base_model.layers:
    layer.trainable = False

In [62]:
batches = gen.flow_from_directory(data_dir+'/train', target_size=target_size, batch_size=batch_size)
batches_val = gen.flow_from_directory(data_dir+'/valid', shuffle=False, target_size=target_size, batch_size=batch_size)

nb_batches = math.ceil(batches.n/batch_size)
nb_batches_val = math.ceil(batches_val.n/batch_size)

y_encode = batches.classes
y_val_encode = batches_val.classes

y = to_categorical(batches.classes)
y_val = to_categorical(batches_val.classes)


Found 8222 images belonging to 120 classes.
Found 2000 images belonging to 120 classes.


In [63]:
model_ft.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])

In [66]:
latest_filename

'/opt/notebooks/data/dog-breed-identification/preprocessed/results/ft_top_0_2017-11-10-20-39.h5'

In [69]:
no_of_epochs = 10
for epoch in range(no_of_epochs):
    print ("Running epoch: %d" % epoch)
    model_ft.fit_generator(batches, 
                    steps_per_epoch=nb_batches, 
                    epochs=1,
                    validation_data=batches_val,
                    validation_steps=nb_batches_val
                   )
    latest_filename = data_dir+'/results/ft_top_%d_%s.h5' %  (epoch,
        datetime.datetime.now().strftime('%Y-%m-%d-%H-%M')
    )
    print(latest_filename)
    model_ft.save(latest_filename)

Running epoch: 0
Epoch 1/1
514/514 [==============================] - 144s - loss: 0.4200 - acc: 0.8665 - val_loss: 0.4487 - val_acc: 0.8655
Running epoch: 1
Epoch 1/1
514/514 [==============================] - 145s - loss: 0.3734 - acc: 0.8843 - val_loss: 0.4445 - val_acc: 0.8695
Running epoch: 2
Epoch 1/1
514/514 [==============================] - 145s - loss: 0.3594 - acc: 0.8913 - val_loss: 0.4603 - val_acc: 0.8665
Running epoch: 3
Epoch 1/1
514/514 [==============================] - 145s - loss: 0.3250 - acc: 0.9003 - val_loss: 0.4816 - val_acc: 0.8710
Running epoch: 4
Epoch 1/1
514/514 [==============================] - 145s - loss: 0.3124 - acc: 0.9039 - val_loss: 0.5074 - val_acc: 0.8690
Running epoch: 5
Epoch 1/1
514/514 [==============================] - 145s - loss: 0.3072 - acc: 0.9095 - val_loss: 0.5058 - val_acc: 0.8630
Running epoch: 6
Epoch 1/1
514/514 [==============================] - 145s - loss: 0.2872 - acc: 0.9126 - val_loss: 0.5286 - val_acc: 0.8640
Running epoch

KeyboardInterrupt: 

In [76]:
model_ft = load_model(data_dir+'/results/ft_top_3_2017-11-10-20-57.h5')

#### then fine-tune more layers

In [9]:
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

0 input_1
1 block1_conv1
2 block1_conv1_bn
3 block1_conv1_act
4 block1_conv2
5 block1_conv2_bn
6 block1_conv2_act
7 block2_sepconv1
8 block2_sepconv1_bn
9 block2_sepconv2_act
10 block2_sepconv2
11 block2_sepconv2_bn
12 conv2d_1
13 block2_pool
14 batch_normalization_1
15 add_1
16 block3_sepconv1_act
17 block3_sepconv1
18 block3_sepconv1_bn
19 block3_sepconv2_act
20 block3_sepconv2
21 block3_sepconv2_bn
22 conv2d_2
23 block3_pool
24 batch_normalization_2
25 add_2
26 block4_sepconv1_act
27 block4_sepconv1
28 block4_sepconv1_bn
29 block4_sepconv2_act
30 block4_sepconv2
31 block4_sepconv2_bn
32 conv2d_3
33 block4_pool
34 batch_normalization_3
35 add_3
36 block5_sepconv1_act
37 block5_sepconv1
38 block5_sepconv1_bn
39 block5_sepconv2_act
40 block5_sepconv2
41 block5_sepconv2_bn
42 block5_sepconv3_act
43 block5_sepconv3
44 block5_sepconv3_bn
45 add_4
46 block6_sepconv1_act
47 block6_sepconv1
48 block6_sepconv1_bn
49 block6_sepconv2_act
50 block6_sepconv2
51 block6_sepconv2_bn
52 block6_sepcon

In [77]:
# we chose to train the top 2 xception blocks, i.e. we will freeze
# the first 115 layers and unfreeze the rest:
for layer in base_model.layers[:116]:
   layer.trainable = False
for layer in base_model.layers[116:]:
   layer.trainable = True

In [79]:
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
model_ft.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
                   
no_of_epochs = 10
histories = []
for epoch in range(no_of_epochs):
    print ("Running epoch: %d" % epoch)
    hist = model_ft.fit_generator(batches, 
                    steps_per_epoch=nb_batches, 
                    epochs=1,
                    validation_data=batches_val,
                    validation_steps=nb_batches_val
                   )
    histories.append(hist)
    latest_filename = data_dir+'/results/ft_%d_%s.h5' %  (epoch,
        datetime.datetime.now().strftime('%Y-%m-%d-%H-%M')
    )
    print(latest_filename)
    model_ft.save(latest_filename)

Running epoch: 0
Epoch 1/1
514/514 [==============================] - 146s - loss: 0.2846 - acc: 0.9123 - val_loss: 0.4808 - val_acc: 0.8730
/opt/notebooks/data/dog-breed-identification/preprocessed/results/ft_0_2017-11-10-21-17.h5
Running epoch: 1
Epoch 1/1
514/514 [==============================] - 145s - loss: 0.2860 - acc: 0.9112 - val_loss: 0.4744 - val_acc: 0.8720
/opt/notebooks/data/dog-breed-identification/preprocessed/results/ft_1_2017-11-10-21-20.h5
Running epoch: 2
Epoch 1/1
514/514 [==============================] - 145s - loss: 0.2878 - acc: 0.9120 - val_loss: 0.4758 - val_acc: 0.8740
/opt/notebooks/data/dog-breed-identification/preprocessed/results/ft_2_2017-11-10-21-22.h5
Running epoch: 3
Epoch 1/1
514/514 [==============================] - 145s - loss: 0.2743 - acc: 0.9145 - val_loss: 0.4703 - val_acc: 0.8715
/opt/notebooks/data/dog-breed-identification/preprocessed/results/ft_3_2017-11-10-21-24.h5
Running epoch: 4
Epoch 1/1
514/514 [==============================] - 14

### predict

In [86]:
batches_test = gen.flow_from_directory(data_dir+'/test', shuffle=False, target_size=target_size, batch_size=batch_size)

Found 10357 images belonging to 1 classes.


In [87]:
nb_batches_test = math.ceil(batches_test.n/batch_size)

In [88]:
pred = model_ft.predict_generator(batches_test, steps=nb_batches_test, verbose=1)

648/648 [==============================] - 160s   


In [89]:
test_ids = [f.split('/')[1].split('.')[0] for f in batches_test.filenames]

In [90]:
subm=pd.DataFrame(np.hstack([np.array(test_ids).reshape(-1, 1), pred]))
labels = pd.read_csv(data_dir+'/labels.csv')
cols = ['id']+sorted(labels.breed.unique())
subm.columns = cols

In [91]:
description = 'xception_data_finetune_more_layers'
submission_file_name = data_dir+'/results/%s_%s.csv' % (description,
                                                        datetime.datetime.now().strftime('%Y-%m-%d-%H-%M')
                                                       )
subm.to_csv(submission_file_name, index=False)

### submit

In [48]:
!kg config -u $KAGGLE_USER -p $KAGGLE_PW -c $competition_name

In [49]:
!kg submit $submission_file_name -m $description

0.39939
